In [1]:
const datastore = require('@lumino/datastore');
const TextField = datastore.TextField;

## Manual exploration

In [2]:
field = new TextField({
  description: 'A test text field.'
});
value = field.createValue()
metadata = field.createMetadata()

{ ids: [], cemetery: {} }

In [18]:
let { value, change, patch } = field.applyUpdate({
    previous: value,
    update: {
        index: 3,
        remove: 0,
        text: 'd'
    },
    metadata,
    version: 4,
    storeId: 1
});


耀      1
 0
lower 140737488355328 1 1
upper 281474976710656 0 0
[ 211106232532992 ]


In [19]:
2**48

281474976710656

In [20]:
value.length

4

In [21]:
value

'cbad'

In [22]:
metadata

{
  ids: [
    ' \u0000\u0000\u0000\u0000\u0003\u0000\u0001',
    '䀀\u0000\u0000\u0000\u0000\u0002\u0000\u0001',
    '耀\u0000\u0000\u0000\u0000\u0001\u0000\u0001',
    '쀀\u0000\u0000\u0000\u0000\u0004\u0000\u0001'
  ],
  cemetery: {}
}

## Random updates

In [2]:
// Random int inclusive of both endpoints
function randomInt(low, high) {
    return low + Math.floor(Math.random() * (high-low+1));
}

In [3]:
randomInt(3, 10)

3

In [4]:
function poissonSample(lambda) {
    let L = Math.exp(-lambda)
    let k = 0;
    let p = 1;
    while (p > L) {
        k = k + 1;
        p = p*Math.random()
    }
    return k-1;
}

In [5]:
poissonSample(10)

10

In [6]:
function positivePoissonSample(lambda) {
    let result = 0;
    while (result === 0) {
        result = poissonSample(lambda)
    }
    return result;
}

In [7]:
positivePoissonSample(2)

3

In [8]:
function randomUpdate(field, version, previous, metadata) {
//     console.log(field, version, previous, metadata);
    let index = randomInt(0, previous.length+1);
    let remove = 0;
    let text = 'A'.repeat(positivePoissonSample(100));
    let newVersion = version + 1;
    let { value } = field.applyUpdate({
        previous,
        update: {
            index,
            remove,
            text
        },
        metadata,
        version: newVersion,
        storeId: 1
    });
    return {newVersion: newVersion, newValue: value};
}

In [359]:
randomField = new TextField({
  description: 'A test text field.'
});
newValue = randomField.createValue();
newVersion = 1;
randomMetadata = randomField.createMetadata();

{ ids: [], cemetery: {} }

In [404]:
let { newVersion, newValue } = randomUpdate(randomField, newVersion, newValue, randomMetadata)

lower 23817113413977 5 1
upper 24899709478249 5 1
[
  23884775667994, 23952437922011,
  24020100176028, 24087762430045,
  24155424684062, 24223086938079,
  24290749192096, 24358411446113,
  24426073700130, 24493735954147,
  24561398208164, 24629060462181,
  24696722716198, 24764384970215,
  24832047224232
]


In [405]:
newVersion

13

In [406]:
newValue

'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [ ]:
randomMetadata

In [410]:
randomMetadata.ids[0].length

8

In [9]:
function metadataSize(metadata) {
    counts = {}
    metadata.ids.forEach((item) => {
        L = item.length/8;
        if (counts[L]) {
            counts[L] = counts[L]+1;
        } else {
            counts[L] = 1;
        }
    });
    return counts;
}

In [10]:
function benchmarkUpdates(N) {
    let randomField = new TextField();
    let newValue = randomField.createValue();
    let newVersion = 1;
    let randomMetadata = randomField.createMetadata();
    for (i=0; i<N; i++) {
        result = randomUpdate(randomField, newVersion, newValue, randomMetadata);
        newVersion = result.newVersion;
        newValue = result.newValue;
    }
    let total = 0;
    let counts = metadataSize(randomMetadata)
    for (const k in counts) {
        total += parseInt(k)*counts[k];
    }
    MB = total*16/1e6;
    return {MB, total, length: newValue.length, counts};
}

In [11]:
2**10

1024

Old algorithm:

In [12]:
benchmarkUpdates(2**12)

{
  MB: 21.772496,
  total: 1360781,
  length: 409031,
  counts: {
    '1': 5251,
    '2': 71895,
    '3': 166175,
    '4': 121097,
    '5': 38962,
    '6': 5540,
    '7': 111
  }
}

New algorithm:

In [17]:
benchmarkUpdates(2**12)

{
  MB: 17.23528,
  total: 1077205,
  length: 410543,
  counts: {
    '1': 155248,
    '2': 71937,
    '3': 67727,
    '4': 51863,
    '5': 33216,
    '6': 18878,
    '7': 7157,
    '8': 2863,
    '9': 1441,
    '10': 213
  }
}

In [16]:
20.8/6.6

3.151515151515152